In [1]:
import pygame as pg
import sys
import time
import random
import numpy as np
import pickle
import os
import neat

pg.font.init()
DRAW_LINES = False

STAT_FONT = pg.font.SysFont("comicsans", 50)
END_FONT = pg.font.SysFont("comicsans", 70)

clock=pg.time.Clock()
sat=pg.time.Clock()
brojac = 0
WIN_SIRINA=800
WIN_VISINA=650
PTICE_SLIK=np.array([])
rem=[]
for x in range(0,15):
	PTICE_SLIK = np.append(PTICE_SLIK,pg.transform.scale((pg.image.load(os.path.join("D:\\voja\\Python\\Projekti\\FlappyBird\\pravatica\\F"+str(x)+".gif"))),(90,90)))

SIPKA_SLIKA = pg.transform.scale((pg.image.load(os.path.join("D:\\voja\\Python\\Projekti\\FlappyBird\\Cevka.png"))),(50,650))
SIPKA_OBRNUTA = pg.transform.flip(SIPKA_SLIKA,False,True)
BG =pg.image.load(os.path.join("D:\\voja\\Python\\Projekti\\FlappyBird\\prva.png"))
BG_rotiran = pg.transform.flip(BG,True,False)
win= pg.display.set_mode((WIN_SIRINA,WIN_VISINA))
gen=0
run=True


class Ptica:

	IMGS=PTICE_SLIK
	pg.init()
	ROTACIJA=25
	ROTACIJA_BRZINA=10
	VREME_ANIMACIJE=1
	def __init__(self, x , y):
		self.x=x
		self.y=y
		self.zakrivljenje = 0
		self.broj_otkucaja = 0
		self.brzina=0
		self.visina=self.y
		self.brojac_slika= 0
		self.slika = self.IMGS[0]
		self.slikadva = self.IMGS[14]

	def skok(self):
		self.brzina = -21.5
		self.broj_otkucaja=0
		self.visina = self.y

	def kretanje(self):

		self.broj_otkucaja+=1

		d= self.brzina*self.broj_otkucaja+4*self.broj_otkucaja**2
		if d>=19:
			d =19
		if d<=0:
			d-=2

		self.y=self.y+d
		if d<0 or self.y<self.visina+90:
			if self.zakrivljenje<self.ROTACIJA:
				self.zakrivljenje=self.ROTACIJA
		else:
			if self.zakrivljenje>-90:
				self.zakrivljenje-=self.ROTACIJA_BRZINA

	def kontrola_kretanja(self,tica):
		for event in pg.event.get():
			if event.type==pg.KEYDOWN:
				if event.key==pg.K_UP:
					tica.skok()
					
			if event.type==pg.QUIT:
				pg.quit()

					
		
	def crtanje_tice(self,win):
		self.brojac_slika+=1
		if self.brojac_slika<= self.VREME_ANIMACIJE:
			self.slika=self.IMGS[0]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*2:
			self.slika=self.IMGS[1]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*3:
			self.slika=self.IMGS[2]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*4:
			self.slika=self.IMGS[3]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*5:
			self.slika=self.IMGS[4]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*6:
			self.slika=self.IMGS[5]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*7:
			self.slika=self.IMGS[6]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*8:
			self.slika=self.IMGS[7]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*9:
			self.slika=self.IMGS[8]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*10:
			self.slika=self.IMGS[9]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*11:
			self.slika=self.IMGS[10]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*12:
			self.slika=self.IMGS[11]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*13:
			self.slika=self.IMGS[12]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*14:
			self.slika=self.IMGS[13]
		elif self.brojac_slika<= self.VREME_ANIMACIJE*15:
			self.slika=self.IMGS[14]
			self.brojac_slika=0
		#if self.zakrivljenje<=-80:
			#self.slika=self.IMGS[1]
			#self.brojac_slika = self.VREME_ANIMACIJE*1
		#rotated_image=pg.transform.rotate(self.slika,self.zakrivljenje)
		#new_rect=rotated_image.get_rect(center=self.slika.get_rect(topleft=(self.x,self.y)).center)    AKO ZELIM ROTACIJU
		#win.blit(rotated_image,new_rect.topleft)
		win.blit(self.slika,((self.x,self.y)))


	def maska(self):
		return pg.mask.from_surface(self.slika)





class Pozadina:
	BRZINA_EKRANA=1
	def __init__(self):
		self.x1=0
		self.x2=self.x1+800
	def kretanje_ekrana(self,pozadina):
		self.x1-=self.BRZINA_EKRANA
		self.x2-= self.BRZINA_EKRANA
		if (self.x1==-800):
			self.x1=800
		if (self.x2==-800):
			self.x2=800
		
	def crtanje_ekrana(self,win):
		win.blit(BG,(self.x1,0)), win.blit(BG_rotiran,(self.x2,0))


class Sipka:
	def __init__(self,poz_x):
		self.visina_gornje = 0
		self.nasumicna_visina_donje=0
		self.visina=0
		self.dodaj_sipku = False
		self.prosao = False
		self.poz_x=poz_x
		self.SIPKA = SIPKA_SLIKA
		self.OBRNUTA=SIPKA_OBRNUTA
		self.prolazni_prostor = 195
		self.brzina_sipki=6
		#self.niz_sipki = list(range(1,10))#####visina i sirina 3690 x 960        WIN_SIRINA=800     WIN_VISINA=650
		#self.nasumicna_visina_donje=0
		self.nasumicna_visina_donje = random.randint(171,650)
		self.visina_gornje = self.nasumicna_visina_donje-self.prolazni_prostor-self.OBRNUTA.get_height()
		
	def kretanje_sipke(self):
		self.poz_x -= self.brzina_sipki
	

	def crtanje_sipke(self,win):

		win.blit(self.SIPKA,(self.poz_x,self.nasumicna_visina_donje))
		win.blit(self.OBRNUTA,(self.poz_x,self.visina_gornje))

	def sudar(self,tica):
		tica_mask = tica.maska()
		gornja_sipka_mask = pg.mask.from_surface(self.OBRNUTA)
		donja_sipka_mask = pg.mask.from_surface(self.SIPKA)
		gornji_offset = (self.poz_x-tica.x, self.visina_gornje-round(tica.y))
		donji_offset = (self.poz_x - tica.x, self.nasumicna_visina_donje-round(tica.y))
		b_tacka = tica_mask.overlap(donja_sipka_mask,donji_offset)
		t_tacka = tica_mask.overlap(gornja_sipka_mask,gornji_offset)
		if b_tacka or t_tacka:
			return True
		return False
	def prosao(self,tica):
		if self.poz_x+self.SIPKA.get_width()==tica.x:
			self.prosao = True








	
def draw_window(win,pozadina,tice,sipka,gen,sipka_posmatrana,score):
	pozadina.crtanje_ekrana(win)
	pozadina.kretanje_ekrana(win)
	if gen == 0:
		gen = 1
	
	#tica.kontrola_kretanja(tica)
	

	for sip in sipka:
		sip.crtanje_sipke(win)
		for tica in tice:
			tica.crtanje_tice(win)
			pg.draw.line(win, (255,0,0), (tica.x+tica.slika.get_width()/2, tica.y + tica.slika.get_height()/2), (sipka[sipka_posmatrana].poz_x , sipka[sipka_posmatrana].nasumicna_visina_donje), 5)
			pg.draw.line(win, (255,0,0), (tica.x+tica.slika.get_width()/2, tica.y + tica.slika.get_height()/2), (sipka[sipka_posmatrana].poz_x, sipka[sipka_posmatrana].visina_gornje+650), 5)
				

	
	
	score_label = STAT_FONT.render("Score: " + str(score),1,(255,255,255))
	win.blit(score_label, (WIN_SIRINA - score_label.get_width() - 15, 10))
			
				
	score_label = STAT_FONT.render("Gens: " + str(gen-1),1,(255,255,255))
	win.blit(score_label, (10, 10))
			
			    # alive
	score_label = STAT_FONT.render("Zive: " + str(len(tice)),1,(255,255,255))
	win.blit(score_label, (10, 50))
	pg.display.update()
		
	
	

def eval_genomes(genomes, config):
	global gen
	gen+=1
	nets = []
	tice = []
	ge = []
	for genome_id, genome in genomes:
		genome.fitness = 0
		net = neat.nn.FeedForwardNetwork.create(genome, config)
		nets.append(net)
		tice.append(Ptica(40,300))
		ge.append(genome)

	
	score=0
	run=True
	win= pg.display.set_mode((WIN_SIRINA,WIN_VISINA))
	pozadina = Pozadina()
	sipka=[Sipka(800)]
	while run and len(tice)>0:
		if score>50:
			break
		sat.tick(30)
		for event in pg.event.get():
			if event.type==pg.QUIT:
				run=False
			if event.type==pg.KEYDOWN:
				if event.key==pg.K_q:
					pg.quit()
					quit()
					break
		pozadina.crtanje_ekrana(win)
		pozadina.kretanje_ekrana(win)
		sipka_posmatrana=0
		if len(tice)>0:
			if len(sipka)>0 and tice[0].x>sipka[0].poz_x+sipka[0].SIPKA.get_width():
				sipka_posmatrana=0
		for x, tica in enumerate (tice):
			ge[x].fitness+=0.1
			tica.kretanje()
			output = nets[tice.index(tica) ].activate((tica.y, abs(tica.y - sipka[sipka_posmatrana].visina_gornje), abs(tica.y - sipka[sipka_posmatrana].nasumicna_visina_donje)))
			if output[0]>0:																																								#OVDE NESTO JEBAVAM
				tica.skok()
		
		rem=[]
		dodaj_sipku=False
		for sipku in sipka:
			for tica in tice:
				if sipku.sudar(tica) == True:
					ge[tice.index(tica)].fitness-=4
					nets.pop(tice.index(tica))
					ge.pop(tice.index(tica))
					tice.pop(tice.index(tica))
					
					
			sipku.kretanje_sipke()
					

			if sipku.poz_x + sipku.SIPKA.get_width()-5<0:
				rem.append(sipku)

			if not sipku.prosao and sipku.poz_x<tica.x:
				sipku.prosao=True
				dodaj_sipku=True
				
		if dodaj_sipku:
			score+=1
			for genome in ge:
				genome.fitness+=1
			sipka.append(Sipka(800))
		for r in rem:
			sipka.remove(r)
		for tica in tice:
			if tica.y + tica.slika.get_height() - 20 >= 650 or tica.y < -50:
				ge[tice.index(tica)].fitness-=4
				

				nets.pop(tice.index(tica))
				ge.pop(tice.index(tica))
				tice.pop(tice.index(tica))
	
		draw_window(win,pozadina,tice,sipka,gen,sipka_posmatrana,score)

def run(config_file):
    
    
    

    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #p.add_reporter(neat.Checkpointer(5))

    # Run for up to 50 generations.
    winner = p.run(eval_genomes, 50)

    # show final stats
    print('\nBest genome:\n{!s}'.format(winner))
	 


if __name__ =='__main__':
	particija=os.path.dirname("D:\\voja\\Python\\Projekti\\FlappyBird\\")
	configfile = os.path.join(particija, "configuration.txt")
	run(configfile)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html

 ****** Running generation 0 ****** 



<ipython-input-1-f949d56e4513>:215: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pg.draw.line(win, (255,0,0), (tica.x+tica.slika.get_width()/2, tica.y + tica.slika.get_height()/2), (sipka[sipka_posmatrana].poz_x , sipka[sipka_posmatrana].nasumicna_visina_donje), 5)
<ipython-input-1-f949d56e4513>:216: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pg.draw.line(win, (255,0,0), (tica.x+tica.slika.get_width()/2, tica.y + tica.slika.get_height()/2), (sipka[sipka_posmatrana].poz_x, sipka[sipka_posmatrana].visina_gornje+650), 5)
<ipython-input-1-f949d56e4513>:126: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  win

error: display Surface quit